<a href="https://colab.research.google.com/github/Andriansyah2501/Proyek_Analisis_Sentimen/blob/main/Analisis_Sentiment_skema_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install scikit-learn

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [4]:
from google.colab import drive
drive.mount('/content/drive')

# Memuat dataset CSV dari hasil clustering
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset sentimen/dataset_review_makanan_bergizi_3200.csv')

# Cek 5 baris pertama buat pastiin data ke-load
print("5 Baris Pertama Dataset:")
print(df.head())

# Cek info dataset (kolom, tipe data, dll.)
print("\nInfo Dataset:")
print(df.info())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
5 Baris Pertama Dataset:
                                              review  rating
0                Makanannya sangat enak dan bergizi.       5
1          Program makan gratis ini sangat membantu.       4
2  Saya tidak terlalu suka dengan menu yang diber...       2
3          Pelayanannya lambat, tapi makanannya oke.       3
4   Sangat bermanfaat untuk masyarakat kurang mampu.       5

Info Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3200 non-null   object
 1   rating  3200 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 50.1+ KB
None


In [5]:
# Label sentimen dari skor rating
def label_sentiment(score):
    if score >= 4:
        return 'positif'
    elif score == 3:
        return 'netral'
    else:
        return 'negatif'

df['sentiment'] = df['rating'].apply(label_sentiment)
print("Distribusi sentimen:\n", df['sentiment'].value_counts())

# TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['review'].astype(str)).toarray()

# Label encoding
le = LabelEncoder()
y = le.fit_transform(df['sentiment'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluasi
y_pred = model.predict(X_test)
print(f"\n🎯 Akurasi Testing: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

# Fungsi prediksi
def prediksi_sentimen(kalimat):
    vector = vectorizer.transform([kalimat]).toarray()
    hasil = model.predict(vector)
    return le.inverse_transform(hasil)[0]

# Contoh prediksi
contoh = "Program ini sangat membantu dan makanannya enak"
print("\n🧠 Prediksi untuk contoh kalimat:")
print(f"Kalimat: {contoh}")
print(f"Sentimen: {prediksi_sentimen(contoh)}")

Distribusi sentimen:
 sentiment
positif    1963
negatif     648
netral      589
Name: count, dtype: int64

🎯 Akurasi Testing: 100.00%

📊 Classification Report:
               precision    recall  f1-score   support

     negatif       1.00      1.00      1.00       129
      netral       1.00      1.00      1.00       118
     positif       1.00      1.00      1.00       393

    accuracy                           1.00       640
   macro avg       1.00      1.00      1.00       640
weighted avg       1.00      1.00      1.00       640


🧠 Prediksi untuk contoh kalimat:
Kalimat: Program ini sangat membantu dan makanannya enak
Sentimen: positif
